<p>Euler's totient function, $\phi(n)$ [sometimes called the phi function], is defined as the number of positive integers not exceeding $n$ which are relatively prime to $n$. For example, as $1$, $2$, $4$, $5$, $7$, and $8$, are all less than or equal to nine and relatively prime to nine, $\phi(9)=6$.</p>
<div class="center">
<table class="grid center"><tr><td><b>$n$</b></td>
<td><b>Relatively Prime</b></td>
<td><b>$\phi(n)$</b></td>
<td><b>$n/\phi(n)$</b></td>
</tr><tr><td>2</td>
<td>1</td>
<td>1</td>
<td>2</td>
</tr><tr><td>3</td>
<td>1,2</td>
<td>2</td>
<td>1.5</td>
</tr><tr><td>4</td>
<td>1,3</td>
<td>2</td>
<td>2</td>
</tr><tr><td>5</td>
<td>1,2,3,4</td>
<td>4</td>
<td>1.25</td>
</tr><tr><td>6</td>
<td>1,5</td>
<td>2</td>
<td>3</td>
</tr><tr><td>7</td>
<td>1,2,3,4,5,6</td>
<td>6</td>
<td>1.1666...</td>
</tr><tr><td>8</td>
<td>1,3,5,7</td>
<td>4</td>
<td>2</td>
</tr><tr><td>9</td>
<td>1,2,4,5,7,8</td>
<td>6</td>
<td>1.5</td>
</tr><tr><td>10</td>
<td>1,3,7,9</td>
<td>4</td>
<td>2.5</td>
</tr></table></div>
<p>It can be seen that $n = 6$ produces a maximum $n/\phi(n)$ for $n\leq 10$.</p>
<p>Find the value of $n\leq 1\,000\,000$ for which $n/\phi(n)$ is a maximum.</p>

In [1]:
'''
Resources:
- https://en.wikipedia.org/wiki/Coprime_integers
- https://en.wikipedia.org/wiki/Euler%27s_totient_function
- https://en.wikipedia.org/wiki/Euclidean_algorithm
'''

# GCD Algorithm
#def gcd(a: int, b: int) -> int:
#    return a if b == 0 else gcd(b, a % b)

gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)

if __name__ == "__main__":
    print("{}".format(gcd(2, 4)))

2


In [2]:
# MAIN PROGRAM

'''
TODO:
- will give correct result, but algorithm is too slow for large N

- using a 2D lookup table: gcd(a, b) -> {x:{...}, ..., a:{x: c, b: n}, ...}
does not speed up the process either, as shown below. There are too many redundant
entries for consecutive and adjacent n values. A more mathematical and ultimately simpler
approach will be used next to solve the problem.
'''

MAX_N: int = 2000

#gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)
gcd_cache: dict = {}

def gcd(a: int, b: int) -> int:
    '''
    - # gcd: int = lambda a, b: a if b == 0 else gcd(b, a % b)
    - # (recursive) Euclidean algorithm to compute GCD
    
    # Ex. n = 8, thus coprimes of 8 and [1,8] are: 1, 3, 5, 7

    # table[a][b] -> gcd(a, b)
    # table can list all coprimes OR show the count of coprimes
    # note that gcd(a, b) = gcd(b, a)

    # dic: dict = {"2": {"1": 1, "2": 2, "3": 1, "14": 2}, "3": {"1": 1, "2": 1, "23": 1}, ...}

    - "2": {"3": x} == "3": {"2": x} so when checking table, let [i, j] be [min(a, b), max(a, b)]
    '''
    # check lookup table
    min_val: int = min(a, b)
    max_val: int = max(a, b)
    if min_val in gcd_cache and max_val in gcd_cache[min_val]:
        return gcd_cache[min_val][max_val]
    
    # compute gcd
    ret_val: int = a if b == 0 else gcd(b, a % b)
    
    # cache the new value and return
    if min_val not in gcd_cache:
        gcd_cache.update({min_val: {}})
    if max_val not in gcd_cache[min_val]:
        gcd_cache[min_val].update({min_val: {max_val: ret_val}})
    return ret_val

if __name__ == "__main__":
    max_n: int = 0
    ret_frac: float = 0.0
    for i in range(2, MAX_N):
        coprime_count: int = 0
        for j in range(1, i + 1):
            if gcd(i, j) == 1:
                coprime_count += 1
        #print("n = {0}, phi(n) = {1}, n / phi(n) = {2}".format(i, coprime_count, float(i / coprime_count)))

        #ret_frac = max(ret_frac, float(i / coprime_count))
        if float(i / coprime_count) > ret_frac:
            ret_frac = float(i / coprime_count)
            max_n = i
    print("\n[n, max(n / phi(n))]: [{0}, {1}]".format(max_n, ret_frac))


[n, max(n / phi(n))]: [210, 4.375]


In [3]:
# NEW APPROACH

'''
- note, this approach requires an access to prime numbers.
The calculation of n/phi(n) produces a higher ratio when phi(n) is minimal,
indicating that prime numbers are likely to have more coprimes thus reducing
the ratio.

The Totient Function is based on the distribution of distinct primes that divide n
'''

def prime_factors(num: int) -> list:
    '''
    - generates the list of prime factors of a given 
    positive integer num
    '''
    prime_factors: list = []
    for term in range(2, num):
        while num % term == 0:
            if num == 1:
                return
            num = num / term
            prime_factors.append(term)
    return prime_factors

def generate_primes(num: int) -> list:
    '''
    - generates a list of the first num prime numbers
    '''
    primes: list = []
    for n in range(2, num + 1):
        is_prime: bool = True
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                is_prime = False
                break
        if is_prime == True:
            primes.append(n)
    return primes

# Example usage:
if __name__ == "__main__":
    n = 11
    prime_numbers = generate_primes(n)
    print(prime_numbers)

    print(prime_factors(900))

[2, 3, 5, 7, 11]
[2, 2, 3, 3, 5, 5]


In [4]:
# MAIN PROGRAM

# list of distinct primes whose product does not exceed 1000000
LIMIT: int = 1000000
PRIMES: list = generate_primes(100)

if __name__ == "__main__":
    '''
    - including more distinct primes in n will increase
    the desired ratio.
    '''
    n: int = 1
    k: int = 0
    while PRIMES[k] * n <= LIMIT:
        n *= PRIMES[k]
        k += 1
    print("n where max(n / phi(n)): {}".format(n))

n where max(n / phi(n)): 510510
